In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time
import os

2024-10-16 06:29:13.140588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 06:29:13.153734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 06:29:13.171475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 06:29:13.176840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 06:29:13.191376: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def display_image(image):
    fig = plt.figure(figsize=(10,8))
    plt.grid(False)
    plt.axis('off')
    plt.imshow(image)

In [3]:
def draw_bounding_box_on_image(image, ymin, xmin, ymax,
                               xmax, color, font, thickness = 4,
                               display_str_list = ()):
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right,top), (left,top)],width = thickness, fill = color)

    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]

    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = bottom + total_display_str_height
    
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                        fill = color)
        draw.text((left+margin, text_bottom - text_height - margin),
                  display_str, fill = 'black', font = font)
        text_bottom -= text_height - 2 * margin

In [4]:
def draw_boxes(image, boxes, class_names, scores, max_boxes = 10, min_score = 0.1):
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype('arial.ttf',25)
    except IOError:
        print('Font not found, using default font.')
        font = ImageFont.load_default()
        
    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = '{}: {}%'.forma(class_names[i].decode('ascii'),
                                          int(100 * scores[i]))
            color = colors[hash(class_names[i] % len(colors))]
            image_pil = Image.fromarray(np.unit8(image)).convert('RGB')
            draw_bounding_box_on_image(
                image_pil, ymin, xmin,
                ymax, xmax, color,
                font, display_str_list = [display_str])
            np.copyto(image, np.array(image_pil))
        return image

In [7]:
module_handle = 'http://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2'

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2024-10-16 06:39:07.875577: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38716 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 3g.40gb, pci bus id: 0000:e3:00.0, compute capability: 8.0


In [8]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels = 3)
    return img

In [9]:
def run_detector(detector, path):
    img = load_img(path)

    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time= time.time()
    result = detector(converted_img)
    end_time = time.time()

    result = {key:value.numpy() for key, value in result.items()}
    print('이미지에서 찾은 %d개의 객체' %len(result['detection_scores']))
    print('추론한 시간: ', end_time - start_time)

    image_with_boxes = draw_boxes(
        img.numpy(),result['detection_boxes'],
        result['detection_class_entitles'], result['detection_scores']
    )

    display_image(image_with_boxes)